In [1]:
!pip install adamp

  Created wheel for adamp: filename=adamp-0.3.0-py3-none-any.whl size=5998 sha256=1179ecdce684966a273ddce4ba0540c10ecbac97a0bbef4f6136bc3f5a5aa474
  Stored in directory: /root/.cache/pip/wheels/32/c3/7e/c2e3196994b608c3a33bb844e9a3ddddad6692f001a04bba7a
Successfully built adamp


In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
from tqdm.notebook import tqdm
import wandb
from adamp import AdamP
import time
import random
import os
import sys

In [2]:
transform_train = transforms.Compose(
[
    transforms.Resize((70, 70)),
    transforms.RandomCrop((64, 64)),
    #transforms.RandomHorizontalFlip(),
    #transforms.RandomVerticalFlip(),
    #transforms.RandomRotation(45),
    #transforms.RandomAffine(45),
    #transforms.ColorJitter(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5,), (0.5, 0.5, 0.5))])

transform_test = transforms.Compose(
[
    transforms.Resize((70, 70)),
    transforms.CenterCrop((64, 64)),
    #transforms.RandomHorizontalFlip(),
    #transforms.RandomVerticalFlip(),
    #transforms.RandomRotation(45),
    #transforms.RandomAffine(45),
    #transforms.ColorJitter(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5,), (0.5, 0.5, 0.5))])


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [5]:
train_dataset = datasets.CIFAR10(root='./data',
                                 train=True,
                                 transform=transform_train,
                                download=True)
valid_dataset = datasets.CIFAR10(root='./data',
                                 train=True,
                                 transform=transform_test)
test_dataset = datasets.CIFAR10(root='./data',
                                train=False,
                                transform=transform_test)
                                
batch = 128
seed = 42 # deep thought
epochs = 20
valid_fraction = 0.2
num_workers = 2

Files already downloaded and verified


In [5]:
dataset_size = len(train_dataset)
idx_list = list(range(dataset_size))
np.random.shuffle(idx_list)

split = int(valid_fraction * dataset_size)

train_idx = idx_list[split:]
valid_idx = idx_list[:split]

train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
valid_sampler = torch.utils.data.SubsetRandomSampler(valid_idx)


train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch,
                          num_workers=num_workers,
                          drop_last=True,
                          sampler=train_sampler)


valid_loader = DataLoader(dataset=valid_dataset,
                          batch_size=batch,
                          num_workers=num_workers,
                          drop_last=True,
                          sampler=valid_sampler)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch,
                         num_workers=num_workers,
                         shuffle=False)


/opt/conda/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 2 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [6]:
print(torch.__version__)
import sys
print(sys.version)

1.9.0+cu111
3.8.10 | packaged by conda-forge | (default, May 11 2021, 07:01:05) 
[GCC 9.3.0]


In [7]:
mobilenet_v3_small = torchvision.models.mobilenet_v3_small(pretrained=False)
print(mobilenet_v3_small.classifier)
mobilenet_v3_small.classifier[-1] = torch.nn.Linear(in_features=1024, out_features=10)
model = mobilenet_v3_small.to(device)
params = model.parameters()

Sequential(
  (0): Linear(in_features=576, out_features=1024, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1024, out_features=1000, bias=True)
)


In [8]:
def evaluation(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (features, targets) in tqdm(enumerate(data_loader), total = len(data_loader)):
            features = features.to(device)
            targets = targets.to(device)
            output = model(features)
            
            if isinstance(output, torch.distributed.rpc.api.RRef):
                output = output.local_value()
            
            _, predicts = torch.max(output, 1)
            total += targets.size(0)
            correct += predicts.eq(targets).sum().item()

    accuracy = round(correct/total * 100., 4)
    return accuracy

In [9]:
def epoch_loss(model, data_loader, device):
    model.eval()
    epoch_loss = 0
    total = 0
    
    with torch.no_grad():
        for feature, targets in data_loader:
            features = features.to(device)
            targets = targets.to(device)
            output = model(features)
            
            if isinstance(output, torch.distributed.rpc.api.RRef):
                output = output.local_value()
                
            total += targets.size(0)
            loss = F.cross_entropy(outpus, targets, reduction='sum')
            epoch_loss += loss
    
    return epoch_loss / total 

In [10]:
def train(model, device, epoch, wandb_name,
          train_loader, valid_loader, test_loader, 
          optimizer, scheduler=None, interval=100):

    #wandb.init(project='mobilenetv3-small-3080ti', entity='arguru', name=wandb_name)      
    start_time = time.time()

    
    for e in range(epoch):
        model.train()
        train_correct = 0
        train_total = 0
        epoch_loss_list = []

        for batch_idx, (features, targets) in tqdm(enumerate(train_loader), total=len(train_loader)):
            features = features.to(device)
            targets = targets.to(device)
            optimizer.zero_grad()

            #forward
            output = model(features)
            loss = F.cross_entropy(output, targets)
            epoch_loss_list.append(loss.item())
            
            #back prop
            loss.backward()
            
            #update parameters
            optimizer.step()

            #calculate accuracy
            _, predicts = torch.max(output, 1)
            # batch_accuracy = round(predicts.eq(targets).sum().item() / targets.size(0) * 100., 4)
            train_correct += predicts.eq(targets).sum().item()
            train_total += targets.size(0)

            #logging
            if batch_idx % interval == 0:
                print(f'Epoch: {e+1:03d}/{epoch:03d}' 
                      f'| Batch: {batch_idx:04d}/{len(train_loader):04d}'
                      f' >> Loss: {loss:.04f}'
                      f' [Time: {time.strftime("%Y-%m-%d %I:%M:%S %p", time.localtime())}]') #f'| Batch acc: {batch_accuracy}'
           



        valid_accuracy = evaluation(model, valid_loader, device)
        train_accuracy = round(train_correct/ train_total * 100., 4)
        elapsed_time = (time.time() - start_time)
        print(f'[[Epoch: {e+1:03d}/{epoch:03d}'
              f'>> Train acc:{train_accuracy:.4f}% | Valid acc:{valid_accuracy:.4f}%'
              f'| Elpased time: {elapsed_time:.2f}sec ({elapsed_time/60:.2f}min)]]')
        #wandb.log({'Epoch':e, 'train/loss':round(np.mean(epoch_loss_list), 4), 
                   #'var/accuracy_top-1':round(valid_accuracy, 4), 'learning_rate':scheduler.get_last_lr()[0]})

        
        
        if scheduler is not None:
            scheduler.step()
        
    test_accuracy = evaluation(model, test_loader, device)
    
    total_time = (time.time() - start_time) / 60
    print(f'======Test acc: {test_accuracy:.4f}% [Total time: {total_time:.2f}min]======\n\n')

    
    

In [11]:
def set_seed(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [23]:
set_seed(seed)

In [13]:

lr = 0.001
optimizer = torch.optim.Adam(params, lr=lr)
#optimizer = AdamP(params, lr=lr)
#optimizer = torch.optim.RMSprop(params, lr=lr, momentum=0.9, eps=0.0316, weight_decay=1e-5)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.96)



train(model=model, device=device, epoch=epochs,
      train_loader=train_loader, valid_loader=valid_loader, test_loader=test_loader, 
      optimizer=optimizer, scheduler=scheduler, wandb_name='ADAM/LR=ie-3/B128/VF=.2/STEPLR(step=3+gamma=0.9)')

  0%|          | 0/312 [00:00<?, ?it/s]

Epoch: 001/020| Batch: 0000/0312 >> Loss: 2.3002 [Time: 2022-06-30 03:00:38 PM]
Epoch: 001/020| Batch: 0100/0312 >> Loss: 1.5960 [Time: 2022-06-30 03:00:42 PM]


KeyboardInterrupt: 